In [1]:
from src.utils.trie_index import find_next_word_continuations, get_distinct_tokens_from_trie
trie_index_path = "/data/hdd1/users/akouk/ARM/ARM/assets/feverous/trie_indexes/ngrams_table_level_1_3.marisa"
results = find_next_word_continuations("Aram", trie_index_path)
print("Results for 'Aram':")
print(results)

Results for 'Aram':
[' ,', ' ]', ' khachaturian', ' khatchaturian', ' manukian', '_', '_khachaturian', '_khatchaturian', '_manukian', 'aic', 'aic_language', 'aic_script', 'ais', 'aized', 'ay', 'azd', 'bagh', 'barri', 'e', 'ean', 'eans', 'eans_in_israel', 'id', 'is', 'is_knight', 'is_rami', 'yan']


In [3]:
import transformers
from guidance import models
import torch

MODEL_NAME = "/data/hdd1/users/akouk/ARM/ARM/assets/cache/Qwen2.5-32B-Instruct-Q4_K_M.gguf"
model = models.LlamaCpp(
    MODEL_NAME,
    n_gpu_layers=-1,
    n_ctx=32768,
    echo=False
)

Process Process-2:
Traceback (most recent call last):
  File "/home/akouk/miniconda3/envs/tolis/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/akouk/miniconda3/envs/tolis/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/akouk/miniconda3/envs/tolis/lib/python3.11/site-packages/guidance/metrics/_metrics.py", line 199, in _monitor_fn
    gpu_stats = gpustat.GPUStatCollection.new_query()
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/akouk/miniconda3/envs/tolis/lib/python3.11/site-packages/gpustat/core.py", line 624, in new_query
    N.nvmlShutdown()
  File "/home/akouk/miniconda3/envs/tolis/lib/python3.11/site-packages/pynvml.py", line 2262, in nvmlShutdown
    ret = fn()
          ^^^^
KeyboardInterrupt


In [4]:
def configure_list_size(string_list, n):
    if not string_list:
        return []
    if len(string_list) <= n:
        return string_list
    max_s_len = 0
    for s_item in string_list:
        if len(s_item) > max_s_len:
            max_s_len = len(s_item)
    optimal_trunc_len = 0
    low = 0
    high = max_s_len

    while low <= high:
        mid_len = low + (high - low) // 2
        
        truncated_set = {s_val[:mid_len] for s_val in string_list}
        
        if len(truncated_set) <= n:
            optimal_trunc_len = mid_len
            low = mid_len + 1 
        else:
            high = mid_len - 1
            
    final_truncated_set = {s_val[:optimal_trunc_len] for s_val in string_list}
    return list(final_truncated_set)

In [13]:
from src.utils.trie_index import find_next_word_continuations
from src.retrieval.base import RetrievalResult
from src.retrieval.bm25 import PyseriniBM25Retriever
from typing import List
import guidance
from guidance import gen, select
from typing import List
from sentence_transformers import SentenceTransformer, util
import torch
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpBinary
from pulp.apis import PULP_CBC_CMD
import re
import numpy as np
import os
import pickle
import faiss

# Define these constants if FaissDenseRetriever class is not imported here
# Otherwise, you can use FaissDenseRetriever.INDEX_FILENAME, etc.
YOUR_FAISS_INDEX_DIR = "/data/hdd1/users/akouk/ARM/ARM/assets/feverous/faiss_indexes/dense_row_UAE-Large-V1" # Path to your FAISS index
FAISS_INDEX_FILENAME = "index.faiss"
FAISS_METADATA_FILENAME = "metadata.pkl"

# Global variables to store pre-loaded FAISS data
loaded_faiss_index = None
loaded_faiss_text_to_original_idx_map = None 
# Path to your FAISS index directory

def initialize_faiss_retrieval_assets_by_text(faiss_folder_path: str = YOUR_FAISS_INDEX_DIR):
    """
    Loads FAISS index and metadata, creating a map from FAISS '_text' content to FAISS internal index.
    This should ideally be called once when the application starts.
    """
    global loaded_faiss_index, loaded_faiss_text_to_original_idx_map

    # Prevent re-initialization if already done
    if loaded_faiss_index is not None and loaded_faiss_text_to_original_idx_map is not None:
        # print("FAISS assets (by text) already initialized.")
        return True

    index_file = os.path.join(faiss_folder_path, FAISS_INDEX_FILENAME)
    metadata_file = os.path.join(faiss_folder_path, FAISS_METADATA_FILENAME)

    if not os.path.exists(index_file) or not os.path.exists(metadata_file):
        print(f"Error: FAISS index or metadata not found in '{faiss_folder_path}'. Pre-computed embeddings unavailable.")
        loaded_faiss_index = None 
        loaded_faiss_text_to_original_idx_map = {}
        return False

    try:
        print(f"Initializing FAISS assets (by text) from: {faiss_folder_path}")
        loaded_faiss_index = faiss.read_index(index_file)
        
        with open(metadata_file, 'rb') as f:
            faiss_metadata_list = pickle.load(f)
        
        temp_text_map = {}
        texts_processed = 0
        for original_idx, meta_item in enumerate(faiss_metadata_list):
            text_content = meta_item.get('_text') 
            if text_content is not None:
                texts_processed += 1
                # If duplicate texts exist in FAISS metadata, this takes the first encountered FAISS index.
                # This should be fine if the text content itself is unique enough for your rows/sentences.
                if text_content not in temp_text_map:
                    temp_text_map[text_content] = original_idx
        
        loaded_faiss_text_to_original_idx_map = temp_text_map
        print(f"FAISS assets (by text) initialized. Index has {loaded_faiss_index.ntotal} vectors.")
        print(f"FAISS text-to-index map created with {len(loaded_faiss_text_to_original_idx_map)} unique text entries from {texts_processed} texts in metadata.pkl.")
        if not loaded_faiss_text_to_original_idx_map and texts_processed > 0:
            print("Warning: FAISS text-to-index map is empty, but texts were found in metadata.pkl. This is unexpected.")
        elif not loaded_faiss_text_to_original_idx_map:
            print("Warning: FAISS text-to-index map is empty. No '_text' fields found or all were None in metadata.pkl.")
        return True

    except Exception as e:
        print(f"Critical error loading FAISS assets (by text): {e}")
        loaded_faiss_index = None
        loaded_faiss_text_to_original_idx_map = {}
        return False
        
        
trie_index_path = "/data/hdd1/users/akouk/ARM/ARM/assets/feverous/trie_indexes/ngrams_table_level_1_3.marisa"
index = PyseriniBM25Retriever()
bm25_index_global = PyseriniBM25Retriever()
bm25_output_folder_global = "/data/hdd1/users/akouk/ARM/ARM/assets/feverous/pyserini_indexes/bm25_row_index" # Your path
embedding_model_global = SentenceTransformer('BAAI/bge-m3')
trie_index_path_global = "/data/hdd1/users/akouk/ARM/ARM/assets/feverous/trie_indexes/ngrams_table_level_1_3.marisa"

COMPATIBILITY_SEMANTIC_WEIGHT = 0.5
COMPATIBILITY_EXACT_WEIGHT = 0.5
MAX_KEYWORD_LENGTH = 20 
MAX_REPHRASES = 5      
BM25_K = 5
RERANK_TOP_K = 5
SCORE_WEIGHT_BM25 = 0.4
SCORE_WEIGHT_EMBEDDING = 0.6
K_MIP_SELECT = 5

def parse_table_object_string(table_str: str) -> dict[str, any]:
    parts = table_str.split(" [SEP] ", 1)
    table_name = parts[0]
    columns_data = {}
    if len(parts) > 1 and parts[1]:
        content_part = parts[1].strip()
        
        if content_part.startswith("[H] "):
            current_attributes_str = content_part[4:]
        else:
            current_attributes_str = content_part
        
        attributes = current_attributes_str.split(" , [H] ")
        
        for attr_str in attributes:
            header_content_split = attr_str.split(":", 1)
            if len(header_content_split) == 2:
                header, content = header_content_split[0].strip(), header_content_split[1].strip()
                if header not in columns_data:
                    columns_data[header] = []
                columns_data[header].append(content)
    return {"name": table_name, "columns": columns_data}

def get_tokens(text: str) -> set[str]:
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    return set(text.split())

def jaccard_similarity(set1: set[str], set2: set[str]) -> float:
    if not set1 and not set2: return 1.0
    if not set1 or not set2: return 0.0
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

def overlap_coefficient(set1: set[str], set2: set[str]) -> float:
    if not set1 and not set2: return 1.0
    if not set1 or not set2: return 0.0
    intersection = len(set1.intersection(set2))
    min_len = min(len(set1), len(set2))
    return intersection / min_len if min_len > 0 else 0.0

def get_semantic_similarity(text1: str, text2: str, model) -> float:
    # Assumes model is a SentenceTransformer model
    emb1 = model.encode(text1, convert_to_tensor=True, show_progress_bar=False)
    emb2 = model.encode(text2, convert_to_tensor=True, show_progress_bar=False)
    return util.cos_sim(emb1, emb2).item()

# --- Compatibility Calculation Functions ---

def calculate_table_table_compatibility(table1_parsed: dict[str, any], table2_parsed: dict[str, any], model) -> float:
    max_col_pair_compatibility = 0.0
    table1_headers = list(table1_parsed["columns"].keys())
    table2_headers = list(table2_parsed["columns"].keys())

    for h1 in table1_headers:
        for h2 in table2_headers:
            header_sem_sim = get_semantic_similarity(h1, h2, model)
            
            col1_value_tokens = set()
            for v_val in table1_parsed["columns"].get(h1, []): col1_value_tokens.update(get_tokens(v_val))
            col2_value_tokens = set()
            for v_val in table2_parsed["columns"].get(h2, []): col2_value_tokens.update(get_tokens(v_val))
            
            exact_val_sim = jaccard_similarity(col1_value_tokens, col2_value_tokens)
            
            col_pair_comp = (COMPATIBILITY_SEMANTIC_WEIGHT * header_sem_sim +
                             COMPATIBILITY_EXACT_WEIGHT * exact_val_sim)
            if col_pair_comp > max_col_pair_compatibility:
                max_col_pair_compatibility = col_pair_comp
    return max_col_pair_compatibility

def calculate_table_passage_compatibility(table_parsed: dict[str, any], passage_text: str, model) -> float:
    max_cell_sentence_compatibility = 0.0
    passage_tokens = get_tokens(passage_text)
    
    for header, cells in table_parsed["columns"].items():
        for cell_content in cells:
            cell_sem_sim = get_semantic_similarity(cell_content, passage_text, model)
            cell_tokens = get_tokens(cell_content)
            exact_val_sim = overlap_coefficient(cell_tokens, passage_tokens)
            
            cell_sentence_comp = (COMPATIBILITY_SEMANTIC_WEIGHT * cell_sem_sim +
                                  COMPATIBILITY_EXACT_WEIGHT * exact_val_sim)
            if cell_sentence_comp > max_cell_sentence_compatibility:
                max_cell_sentence_compatibility = cell_sentence_comp
    return max_cell_sentence_compatibility

def calculate_passage_passage_compatibility(passage1_text: str, passage2_text: str, model) -> float:
    sem_sim = get_semantic_similarity(passage1_text, passage2_text, model)
    passage1_tokens = get_tokens(passage1_text)
    passage2_tokens = get_tokens(passage2_text)
    exact_val_sim = overlap_coefficient(passage1_tokens, passage2_tokens)
    return (COMPATIBILITY_SEMANTIC_WEIGHT * sem_sim +
            COMPATIBILITY_EXACT_WEIGHT * exact_val_sim)

# --- MIP Solver Function ---
def solve_mip_object_selection(
    candidate_objects: list[dict[str, any]], # This list now contains ALL M unique retrieved objects
    k_select: int,
    model # SentenceTransformer model
) -> list[dict[str, any]]:
    
    num_objects = len(candidate_objects) # M
    if num_objects == 0: return []
    if k_select <= 0 : return [] 
    if k_select > num_objects: k_select = num_objects # Cannot select more than available

    # C_ij calculation (remains the same)
    C = np.zeros((num_objects, num_objects))
    for i in range(num_objects):
        for j in range(i + 1, num_objects): 
            obj_i = candidate_objects[i]
            obj_j = candidate_objects[j]
            comp_val = 0.0
            if obj_i['source_type'] == 'table' and obj_j['source_type'] == 'table':
                comp_val = calculate_table_table_compatibility(obj_i['parsed_content'], obj_j['parsed_content'], model)
            elif obj_i['source_type'] == 'table' and obj_j['source_type'] == 'passage':
                comp_val = calculate_table_passage_compatibility(obj_i['parsed_content'], obj_j['text'], model)
            elif obj_i['source_type'] == 'passage' and obj_j['source_type'] == 'table':
                comp_val = calculate_table_passage_compatibility(obj_j['parsed_content'], obj_i['text'], model)
            elif obj_i['source_type'] == 'passage' and obj_j['source_type'] == 'passage':
                comp_val = calculate_passage_passage_compatibility(obj_i['text'], obj_j['text'], model)
            C[i, j] = C[j, i] = comp_val 
    
    # R_i are pre-calculated and passed in candidate_objects[i]['relevance_score_R_i']
    R = [obj['relevance_score_R_i'] for obj in candidate_objects]

    prob = LpProblem("ObjectSelectionMIP", LpMaximize)
    b = [LpVariable(f"b_{i}", cat=LpBinary) for i in range(num_objects)]
    c_vars = {} 
    for i in range(num_objects):
        for j in range(i + 1, num_objects):
            c_vars[(i,j)] = LpVariable(f"c_{i}_{j}", cat=LpBinary)

    objective = lpSum(R[i] * b[i] for i in range(num_objects))
    if c_vars: # Add compatibility term only if there are pairs to consider
        objective += lpSum(C[i][j] * c_vars[(i,j)] for i in range(num_objects) for j in range(i+1, num_objects))
    prob += objective

    prob += lpSum(b[i] for i in range(num_objects)) == k_select 
    
    if k_select > 1 and c_vars: 
         prob += lpSum(c_vars[(i,j)] for i in range(num_objects) for j in range(i+1, num_objects)) <= 2 * (k_select - 1)
    elif k_select == 1 and c_vars: 
         prob += lpSum(c_vars[(i,j)] for i in range(num_objects) for j in range(i+1, num_objects)) == 0

    if c_vars:
        for i in range(num_objects):
            for j in range(i + 1, num_objects):
                prob += 2 * c_vars[(i,j)] <= b[i] + b[j]
    
    prob.solve(PULP_CBC_CMD(msg=0)) 

    selected_objects_indices = [i for i, var_b in enumerate(b) if var_b.value() is not None and var_b.value() > 0.5]
    
    return [candidate_objects[idx] for idx in selected_objects_indices]



@guidance(stateless=False) # Needs to be stateful to use Python logic based on captures
def dynamic_retrieval_guidance(lm, user_query: str):
    if loaded_faiss_index is None or loaded_faiss_text_to_original_idx_map is None:
        print("FAISS assets not pre-initialized, attempting to initialize now (by text)...")
        if not initialize_faiss_retrieval_assets_by_text(): # Uses YOUR_FAISS_INDEX_DIR
            print("Failed to initialize FAISS assets. Pre-computed embeddings will not be used.")
            
    # 1. Decompose query into keywords
    lm += f"""You are given a user question, your task is to decompose the user question into contiguous, non-overlapping substrings that can
cover different information mentioned in the user question. For each substring, generate n-grams that are the most relevant to
the substring. Prefare lengthy n-grams over short ones. Based on the generated relevant n-grams, generate a list of relevant objects, including their names, content, and
connections between these objects. From these candidate objects, you should identify the minimum number of objects that can
be used to answer the user question based on the relevance between the object name, object content and user question as well as
the relevance of the object connections. You should end your response with <>.

User question: What is the birth date of the director of the movie 'Inception' which was released in 2010?
The relevant keywords are birth date | director | movie 'Inception' | released in 2010
The relevant n-grams are birth date (born, date of birth, DOB) | director (directed by, filmmaker, film director) | movie 'Inception' (Inception, film Inception, movie named Inception) | released in 2010 (2010, release year 2010, came out 2010)

Here are the objects that can be relevant:
(...4 objects omitted...)
- Inception was directed by Christopher Nolan.
- Lions live in the jungle.


Here are the objects that are enough to answer the user query:
(...2 objects omitted...)
- Inception was directed by Christopher Nolan.


User question: {user_query}
The relevant keywords are: """
    lm += gen(name='keywords_str', stop='\n')
    keywords_str = lm['keywords_str'].strip()
    keywords = [k.strip() for k in keywords_str.split('|') if k.strip()]

    # 2. Rephrase keywords using dynamic constraints (LLM part with Python logic)
    lm += "\nThe relevant n-grams are"
    all_parsed_ngrams_for_bm25_queries = []
    # ... (N-gram generation loop remains IDENTICAL to your last provided version) ...
    for keyword_idx, keyword in enumerate(keywords):
        if keyword_idx > 0: lm += " |"
        lm += f" {keyword} ("
        ngrams_for_this_keyword_list = []
        current_ngram_being_built_in_python = ""
        keyword_ngram_generation_finished = False
        for rephrase_num in range(MAX_REPHRASES): # MAX_REPHRASES needs to be defined
            first_token_var = f"rephrase_{keyword_idx}_{rephrase_num}_first_token"
            lm += gen(name=first_token_var, max_tokens=1)
            if lm[first_token_var] == ')':
                keyword_ngram_generation_finished = True; break
            current_ngram_being_built_in_python = lm[first_token_var]
            if len(current_ngram_being_built_in_python) < 2 and lm[first_token_var] != ' ':
                second_token_var = f"rephrase_{keyword_idx}_{rephrase_num}_second_token"
                lm += gen(name=second_token_var, max_tokens=1)
                if lm[second_token_var] == ')':
                    if current_ngram_being_built_in_python.strip(): ngrams_for_this_keyword_list.append(current_ngram_being_built_in_python.strip())
                    current_ngram_being_built_in_python = ""; keyword_ngram_generation_finished = True; break
                current_ngram_being_built_in_python += lm[second_token_var]
            for token_step in range(MAX_KEYWORD_LENGTH): # MAX_KEYWORD_LENGTH needs to be defined
                continuations = find_next_word_continuations(current_ngram_being_built_in_python, trie_index_path_global) # trie_index_path_global
                valid_next_strings = continuations + [')', ', ']
                if ' ' not in continuations and (not current_ngram_being_built_in_python or current_ngram_being_built_in_python[-1] != ' '):
                    valid_next_strings.append(' ')
                valid_next_strings = list(set(s for s in valid_next_strings if s and s != '|'))
                if not valid_next_strings: valid_next_strings = [')', ', ', ' ']
                valid_next_strings = configure_list_size(valid_next_strings, 2000) # configure_list_size
                if not valid_next_strings: valid_next_strings = [')']
                current_token_var = f"rephrase_{keyword_idx}_{rephrase_num}_token_{token_step}"
                lm += select(options=valid_next_strings, name=current_token_var)
                selected_token_str = lm[current_token_var]
                if selected_token_str == ')':
                    if current_ngram_being_built_in_python.strip(): ngrams_for_this_keyword_list.append(current_ngram_being_built_in_python.strip())
                    current_ngram_being_built_in_python = ""; keyword_ngram_generation_finished = True; break
                elif selected_token_str == ', ':
                    if current_ngram_being_built_in_python.strip(): ngrams_for_this_keyword_list.append(current_ngram_being_built_in_python.strip())
                    current_ngram_being_built_in_python = ""
                    if rephrase_num < MAX_REPHRASES - 1: lm += " "
                    break 
                else: current_ngram_being_built_in_python += selected_token_str
            if keyword_ngram_generation_finished: break
        if current_ngram_being_built_in_python.strip(): ngrams_for_this_keyword_list.append(current_ngram_being_built_in_python.strip())
        if ngrams_for_this_keyword_list: all_parsed_ngrams_for_bm25_queries.append(" ".join(ngrams_for_this_keyword_list))


    # 3. Retrieve with BM25 (Python logic)
    retrieved_docs_by_bm25_nested: List[List[RetrievalResult]] = []
    if all_parsed_ngrams_for_bm25_queries:
        retrieved_docs_by_bm25_nested = bm25_index_global.retrieve( # bm25_index_global
            nlqs=all_parsed_ngrams_for_bm25_queries,
            output_folder=bm25_output_folder_global, # bm25_output_folder_global
            k=BM25_K # BM25_K needs to be defined
        )
    
    # 4. Collect unique objects (M candidates for MIP)
    unique_retrieved_objects_map = {} # Maps BM25 doc_id to {'doc': RetrievalResult, 'max_bm25_score': float}
    for result_list_for_query in retrieved_docs_by_bm25_nested:
        for res_item in result_list_for_query:
            # Assuming res_item.metadata['id'] is the unique ID from BM25
            doc_id = res_item.metadata.get('id', res_item.object) # Fallback to object text if no id
            if doc_id not in unique_retrieved_objects_map:
                unique_retrieved_objects_map[doc_id] = {'doc': res_item, 'max_bm25_score': res_item.score}
            else:
                unique_retrieved_objects_map[doc_id]['max_bm25_score'] = max(
                    unique_retrieved_objects_map[doc_id]['max_bm25_score'], res_item.score
                )

    if not unique_retrieved_objects_map:
        lm += "\nNo relevant objects were initially retrieved by BM25."
        # ... (rest of the no objects message) ...
        lm += "\nHere are the objects that can be relevant to answer the user query:\nNo relevant objects found.\n"
        lm += "\nHere are the objects that are enough to answer the user query:\nNo objects to select from.\n<>"
        return lm

    # 5. Prepare MIP Input - Calculate R_i using pre-computed FAISS embeddings (text-matching)
    
    # Encode query on-the-fly
    query_embedding_np = embedding_model_global.encode(user_query, convert_to_numpy=True, show_progress_bar=False)
    if query_embedding_np.ndim == 1: query_embedding_np = query_embedding_np.reshape(1, -1)
    faiss.normalize_L2(query_embedding_np) 
    query_embedding_tensor = torch.tensor(query_embedding_np)

    reconstructed_object_embeddings_list = []
    # Store BM25 RetrievalResult objects for which we found FAISS embeddings, to maintain order for similarity scores
    bm25_results_with_faiss_embeddings = [] 

    # Check if FAISS assets are loaded; if not, R_i will be 0 for all (or we could fall back to on-the-fly)
    can_use_faiss_embeddings = (loaded_faiss_index is not None and 
                                loaded_faiss_text_to_original_idx_map is not None and
                                len(loaded_faiss_text_to_original_idx_map) > 0)

    if can_use_faiss_embeddings:
        print("Attempting to use pre-computed FAISS embeddings (by text match)...")
        for bm25_doc_id_key, data_item in unique_retrieved_objects_map.items():
            bm25_object_text = data_item['doc'].object 
            original_faiss_idx = loaded_faiss_text_to_original_idx_map.get(bm25_object_text)
            
            if original_faiss_idx is not None:
                try:
                    if 0 <= original_faiss_idx < loaded_faiss_index.ntotal:
                        reconstructed_emb = loaded_faiss_index.reconstruct(original_faiss_idx)
                        reconstructed_object_embeddings_list.append(reconstructed_emb)
                        bm25_results_with_faiss_embeddings.append(data_item['doc']) # Store the BM25 doc
                    # else: print(f"Debug: FAISS index {original_faiss_idx} out of bounds for text: {bm25_object_text[:30]}")
                except Exception: # Catch-all for reconstruction errors
                    # print(f"Debug: Error reconstructing FAISS emb for text: {bm25_object_text[:30]}")
                    pass # Silently skip if reconstruction fails for an entry
            # else:
                # print(f"Debug: Text not found in FAISS map: {bm25_object_text[:30]}")


    object_embeddings_tensor = torch.empty(0)
    cosine_similarities_scores = [] # List of float scores

    if reconstructed_object_embeddings_list:
        object_embeddings_np_stack = np.array(reconstructed_object_embeddings_list)
        if object_embeddings_np_stack.ndim == 1 and len(reconstructed_object_embeddings_list) == 1:
             object_embeddings_np_stack = object_embeddings_np_stack.reshape(1, -1)
        if object_embeddings_np_stack.size > 0:
            object_embeddings_tensor = torch.tensor(object_embeddings_np_stack)

    if query_embedding_tensor.nelement() > 0 and object_embeddings_tensor.nelement() > 0:
        cosine_similarities_tensor = util.cos_sim(query_embedding_tensor, object_embeddings_tensor)[0]
        cosine_similarities_scores = cosine_similarities_tensor.tolist()

    # Map similarity scores back to the original BM25 unique objects
    # This map uses the same key as unique_retrieved_objects_map (BM25 doc_id or fallback)
    text_to_similarity_score_map = {}
    for i, bm25_doc_found_in_faiss in enumerate(bm25_results_with_faiss_embeddings):
        if i < len(cosine_similarities_scores):
            # Use the object text of the BM25 doc that had a FAISS match as the key
            text_key = bm25_doc_found_in_faiss.object 
            text_to_similarity_score_map[text_key] = (cosine_similarities_scores[i] + 1) / 2.0


    mip_input_candidates = []
    # Iterate through the original unique_retrieved_objects_map to build MIP candidates
    for bm25_doc_id_key, data_item in unique_retrieved_objects_map.items():
        doc_object_text = data_item['doc'].object
        relevance_score_R_i = 0.0 # Default

        if can_use_faiss_embeddings:
            relevance_score_R_i = text_to_similarity_score_map.get(doc_object_text, 0.0)
        else:
            # Fallback: If FAISS pre-computed embeddings are not available/usable,
            # calculate R_i on the fly for this specific object.
            # This part makes it slower if FAISS fails, but keeps it functional.
            # print(f"FAISS pre-computed not used for: {doc_object_text[:30]}. Calculating on-the-fly.")
            obj_emb_tensor = embedding_model_global.encode(doc_object_text, convert_to_tensor=True, show_progress_bar=False)
            if obj_emb_tensor.ndim == 1: obj_emb_tensor = obj_emb_tensor.unsqueeze(0) # Ensure 2D
            if query_embedding_tensor.nelement() > 0 and obj_emb_tensor.nelement() > 0:
                sim = util.cos_sim(query_embedding_tensor, obj_emb_tensor)[0][0].item()
                relevance_score_R_i = (sim + 1) / 2.0
        
        source_str = data_item['doc'].metadata.get('source', 'unknown_source') 
        source_type = 'table' if source_str.startswith('table') else 'passage'
        
        parsed_content_val = None
        if source_type == 'table':
            parsed_content_val = parse_table_object_string(data_item['doc'].object) # parse_table_object_string
        
        mip_input_candidates.append({
            'id': bm25_doc_id_key, # The unique ID from BM25 (or fallback)
            'text': doc_object_text,
            'source_type': source_type,
            'parsed_content': parsed_content_val, 
            'relevance_score_R_i': relevance_score_R_i
        })

    # 6. MIP Solver (Python logic)
    selected_objects_by_mip = []
    if mip_input_candidates:
        selected_objects_by_mip = solve_mip_object_selection( # solve_mip_object_selection
            candidate_objects=mip_input_candidates,
            k_select=K_MIP_SELECT, # K_MIP_SELECT needs to be defined
            model=embedding_model_global # For compatibility calculations within MIP
        )
    
    # 7. Construct LLM Prompt with MIP Results
    lm += "\nHere are the objects that can be relevant to answer the user query:\n"
    if selected_objects_by_mip:
        for obj_data_item in selected_objects_by_mip:
            lm += f"- {obj_data_item['text']}\n"
    else:
        lm += "No specific objects were selected by the solver as most relevant.\n"
    
    lm += "\nHere are the objects that are enough to answer the user query:\n"
    lm += gen(name='final_selected_objects_and_reasoning', stop='<>') 
    lm += "<>"

    return lm

2025-05-17 10:26:34,202 - INFO - Use pytorch device_name: cuda:0
2025-05-17 10:26:34,203 - INFO - Load pretrained SentenceTransformer: BAAI/bge-m3


In [ ]:
user_query = "Aramais Yepiskoposan played for FC Ararat Yerevan, an Armenian football club based in Yerevan during 1986 to 1991."

executed_program = model + dynamic_retrieval_guidance(user_query)

# Print the final generated output
print("\n--- Final LLM Output ---")
print(str(executed_program))

FAISS assets not pre-initialized, attempting to initialize now (by text)...
Initializing FAISS assets (by text) from: /data/hdd1/users/akouk/ARM/ARM/assets/feverous/faiss_indexes/dense_row_UAE-Large-V1
FAISS assets (by text) initialized. Index has 391605 vectors.
FAISS text-to-index map created with 389201 unique text entries from 391605 texts in metadata.pkl.


Retrieving with BM25: 100%|██████████| 6/6 [00:00<00:00, 5661.60it/s]


Attempting to use pre-computed FAISS embeddings (by text match)...

--- Final LLM Output ---
You are given a user question, your task is to decompose the user question into contiguous, non-overlapping substrings that can
cover different information mentioned in the user question. For each substring, generate n-grams that are the most relevant to
the substring. Prefare lengthy n-grams over short ones. Based on the generated relevant n-grams, generate a list of relevant objects, including their names, content, and
connections between these objects. From these candidate objects, you should identify the minimum number of objects that can
be used to answer the user question based on the relevance between the object name, object content and user question as well as
the relevance of the object connections. You should end your response with <>.

User question: What is the birth date of the director of the movie 'Inception' which was released in 2010?
The relevant keywords are birth date | dire

: 